In [2]:
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="vscode")
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

import bar_chart_race as bcr
import ffmpeg
import matplotlib
import folium

In [3]:
from limpar_dados import remover_linhas_colunas_vazias

In [4]:
shark = pd.read_csv(r'C:\Users\carlo\OneDrive\ANALYTICS\ANALYTICS - IRONHACK\ARQUIVOS - ANALYTICS IRONHACK\DESAFIOS_IRONHACK\Shark-Attack\archive\attacks.csv', 
                    sep=',',
                    encoding='latin1',
                    header=0)


In [5]:
shark

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Identificado acima que o arquivo .csv possui várias linhas vazias
# A linha 25722 possui apenas 'xx' na ultima linha
# Várias células vazias no meio do DF
# Coluna Orginal Order apresenta a ordem dos eventos, no entanto sempre acrescentando +1 para o evento mais recente, lista crescente
# 25723 rows × 24 columns


In [7]:
shark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [8]:
shark.describe()

,Year,original order
count,6300.000000,6309.000000
mean,1927.272381,3155.999683
std,281.116308,1821.396206
min,0.000000,2.000000
25%,1942.000000,1579.000000
50%,1977.000000,3156.000000
75%,2005.000000,4733.000000
max,2018.000000,6310.000000


In [9]:
shark.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [10]:
#empty_rows = shark.index[shark.isnull().all(axis=1)]    # Identificar as linhas vazias no DataFrame   
#empty_rows

In [11]:
# Exclui do DataFrame as linhas vazias encontradas acima
shark_no_empty = shark.dropna(how="all").copy(deep=True)    # how neste caso faz a mesma função do all da celula acima, pega todas as linhas nulas neste caso

In [12]:
shark_no_empty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8703 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null  

In [13]:
shark_no_empty

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8698,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8701,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
shark_no_empty.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
             8693,  8694,  8695,  8696,  8697,  8698,  8699,  8700,  8701,
            25722],
           dtype='int64', length=8703)

In [15]:
# Identificar a quantidade de linhas preenchidas com valor válido em cada coluna para analisar distorções
print(shark_no_empty[['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23']].count())


Case Number               8702
Date                      6302
Year                      6300
Type                      6298
Country                   6252
Area                      5847
Location                  5762
Activity                  5758
Name                      6092
Sex                       5737
Age                       3471
Injury                    6274
Fatal (Y/N)               5763
Time                      2948
Species                   3464
Investigator or Source    6285
pdf                       6302
href formula              6301
href                      6302
Case Number.1             6302
Case Number.2             6302
original order            6309
Unnamed: 22                  1
Unnamed: 23                  2
dtype: int64


In [16]:
# Identificado acima que o DataFrame possui divergencia na quantidade de linhas com preenchimento de dados válidos entre as colunas
# Unnamed: 22 com apenas uma celula preenchida e Unnamed: 23 com duas.
# Identifico que a coluna case Number é um outlier com aproximadamente 2400 celulas preenchidas a mais que as demais celulas


In [17]:
# Identificar as linhas preenchidas nas colunas Unnamed: 22 e Unnamed: 23
shark_no_empty.index[shark_no_empty['Unnamed: 22'].notnull()], shark_no_empty.index[shark_no_empty['Unnamed: 23'].notnull()]

(Int64Index([1478], dtype='int64'), Int64Index([4415, 5840], dtype='int64'))

In [18]:
shark_no_empty.loc[[1478, 4415, 5840]]   # Um DF para as linhas das colunas com 1 e 2 células preenchidas para entender quais dados contém

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
1478,2006.05.27,27-May-2006,2006.0,Unprovoked,USA,Hawaii,"North Shore, O'ahu",Surfing,Bret Desmond,M,...,NaN,R. Collier,2006.05.27-Desmond.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.05.27,2006.05.27,4825.0,stopped here,NaN
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


In [19]:
# Exclui as colunas 'Unnamed: 22','Unnamed: 23'
shark_no_empty = shark_no_empty.drop(['Unnamed: 22','Unnamed: 23'], axis=1)

In [20]:
# Mostra as colunas atualizado
shark_no_empty.columns  

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [21]:
shark_no_empty

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8698,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8701,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
counts = shark_no_empty.count(axis=1)   # Conta as linhas com algum preenchimento válido
print(counts)

0        22
1        21
2        21
3        20
4        20
         ..
8698      1
8699      1
8700      1
8701      1
25722     1
Length: 8703, dtype: int64


In [23]:

counts = shark_no_empty.count(axis=1)    # Conta somente as linhas com preenchimento de apenas um caractere válido
print(counts.loc[counts == 1])


6309     1
6310     1
6311     1
6312     1
6313     1
        ..
8698     1
8699     1
8700     1
8701     1
25722    1
Length: 2394, dtype: int64


In [24]:
# Aqui identifica as células da coluna 'Case Number' que estão preenchidas com somente até 2 caracteres e dropa, estas linhas não possuem dados
# Muitas linhas preenchidas com 0 e a ultima preenchida com 'xx' somente todas somente na primeira coluna
shark_no_empty = shark_no_empty.drop(shark_no_empty[shark_no_empty['Case Number'].str.len() <= 2].index)
shark_no_empty

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0


In [25]:
# Temos aqui atualização da quantidade de linhas preenchidas em cada coluna
# Já podemos observar falta de dados pela discrepância apresentada nesta contagem
# Mas observamos também que o DataFrame está ficando mais coeso
celulas_preenchidas = shark_no_empty.count()     
print(celulas_preenchidas.max(), celulas_preenchidas.min())     # Mostra máximo(.max()) e mínimo(.min()) de células que uma coluna tem preenchidas
print(celulas_preenchidas)        #Mostra todas as colunas com a quantidade respectiva de células preendhidas


6302 2948
Case Number               6301
Date                      6302
Year                      6300
Type                      6298
Country                   6252
Area                      5847
Location                  5762
Activity                  5758
Name                      6092
Sex                       5737
Age                       3471
Injury                    6274
Fatal (Y/N)               5763
Time                      2948
Species                   3464
Investigator or Source    6285
pdf                       6302
href formula              6301
href                      6302
Case Number.1             6302
Case Number.2             6302
original order            6302
dtype: int64


In [26]:
# Tratamento coluna Date
shark_no_empty[shark_no_empty['Year'].isnull()]   # Verifica que não possui nenhuma linha não preenchida na coluna Date

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
187,2017.01.08.R,Reported 08-Jan-2017,NaN,Invalid,AUSTRALIA,Queensland,NaN,Spearfishing,Kerry Daniel,M,...,NaN,NaN,Bull shark,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.01.08.R,2017.01.08.R,6116.0
6079,1836.08.19.R,Reported 19-Aug-1836,NaN,Unprovoked,ENGLAND,Cumberland,Whitehaven,Swimming,a boy,M,...,Y,NaN,NaN,"C. Moore, GSAF",1835.08.19.R-Whitehaven.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1836.08.19.R,1836.08.19.R,224.0


In [27]:
# Substitui valor nulo da coluna Year pelo valor correspondente ao ano que estava na coluna Date
shark_no_empty.loc[shark_no_empty['Year'].isnull(),'Year']= shark_no_empty['Date'].str[-4:]
shark_no_empty['Year'].isnull().sum()

0

In [28]:
shark_no_empty.loc[187]

Case Number                                                    2017.01.08.R
Date                                                   Reported 08-Jan-2017
Year                                                                   2017
Type                                                                Invalid
Country                                                           AUSTRALIA
Area                                                             Queensland
Location                                                                NaN
Activity                                                       Spearfishing
Name                                                           Kerry Daniel
Sex                                                                       M
Age                                                                      35
Injury                               No attack, shark made a threat display
Fatal (Y/N)                                                             NaN
Time        

In [29]:
Shark_evol_anual = shark_no_empty.copy(deep=True)
Shark_evol_anual

                        # Pausado shark_no_empty['Date'] = pd.to_datetime(shark_no_empty['Date'])

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0


In [30]:
Shark_evol_anual.loc[(Shark_evol_anual['Year'] == 0)&(Shark_evol_anual['Date'].str[-4:].str.isdigit()),'Year'] = Shark_evol_anual['Date'].str[-4:]
Shark_evol_anual.loc[Shark_evol_anual['Year'].isnull()]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order


In [31]:
# Pausado, estava com o DF errado, anterior
#shark_no_empty.loc[(shark_no_empty['Year'] == 0)&(shark_no_empty['Date'].str[-4:].str.isdigit()),'Year'] = shark_no_empty['Date'].str[-4:]
#shark_no_empty.loc[shark_no_empty['Year'].isnull()]

In [32]:

#provisorio = (Shark_evol_anual['Date'].map(lambda x: False if x.str.contains('B.C.') else True))               #   B.C. - antes de num
#provisorio.value_counts()

In [33]:
#provisorio = Shark_evol_anual['Date'].str.contains('B.C.|b.c.')
#provisorio.value_counts()
#Shark_evol_anual.loc[provisorio, 'Date'].map(lambda x: )            # =True - junto 000

In [34]:
Shark_evol_anual.loc[(Shark_evol_anual['Year'] == 0),'Year'] = Shark_evol_anual.loc[(Shark_evol_anual['Year'] == 0)]['Date'].str.extract('(\d+)')[0]
#shark_no_empty.loc[(shark_no_empty['Year'] == 0)]['Date'].str.extract('(\d+)')[0]

#shark_no_empty.loc[(shark_no_empty['Year'] == 0)]['Date'].str.replace('.*?(\d+).*?',r'\1',regex=True)

In [35]:
Shark_evol_anual.loc[Shark_evol_anual['Year'].isnull()]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
6217,ND.0097,No date,NaN,Unprovoked,USA,Florida,"Key West, Monroe County",Kitesurfing,Paul Menta,M,...,N,NaN,NaN,Internet,ND-0097-PaulMenta.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0097,ND.0097,86.0
6218,ND.0096,No date,NaN,Unprovoked,REUNION,Grand'Anse,Petite-île,yachtsman in a zodiac,NaN,M,...,N,NaN,NaN,G. Van Grevelynghe,ND-0096-Zodiac-Reunion.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0096,ND.0096,85.0
6261,ND.0042,"""Before the war""",NaN,Unprovoked,AUSTRALIA,Torres Strait,Thursday Island?,Free diving,Mortakee,M,...,N,NaN,NaN,Press clipping dated 6/28/1950,ND-0042-Mortakee.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0042,ND.0042,42.0
6271,ND.0032,World War II,NaN,Sea Disaster,PAPUA NEW GUINEA,Between New Ireland & New Britain,St. Georges Channel,Spent 8 days in dinghy,pilot,M,...,N,NaN,NaN,"G.A. Llano in Airmen Against the Sea, p.69; V....",ND-0032-RabaulPilot.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0032,ND.0032,32.0
6272,ND.0031,World War II,NaN,Sea Disaster,PAPUA NEW GUINEA,Madang Province,Off Lae,"Aircraft ditched in the sea, swimming ashore",male,M,...,N,NaN,NaN,"V.M. Coppleson (1962), p.257",ND-0031-Ditched-Aircrat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0031,ND.0031,31.0
6275,ND.0027,No date,NaN,Unprovoked,GREECE,Dodecanese Islands,Symi Island,Sponge diving,Psarofa-gomenes,M,...,N,NaN,NaN,M. N. Kalafatas,ND-0027-Psarofagomenos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0027,ND.0027,28.0
6279,ND.0023,No date,NaN,Unprovoked,SOUTH AFRICA,Western Cape Province,Arniston,Wading,Madelaine Dalton,F,...,N,NaN,NaN,"L. Green in Tavern of the Seas, p.182",ND-0023-Dalton.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0023,ND.0023,24.0
6280,ND.0022,No date,NaN,Unprovoked,AUSTRALIA,NaN,NaN,Pearl diving,Jaringoorli,M,...,N,NaN,NaN,"Adelaide Advertiser, 1/11/1940",ND-0022-Jaringoorli.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0022,ND.0022,23.0
6281,ND.0021,No date,NaN,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Durban,Swimming in pool formed by construction of a w...,Indian boy,M,...,Y,NaN,NaN,"L. Green in South African Beachcomber, p.97",ND-0021-DurbanIndianBoy.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0021,ND.0021,22.0


In [36]:
# Pausado, já havia feito células acima
# Shark_evol_anual = shark_no_empty.copy(deep=True)

In [37]:
Shark_evol_anual

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0
6300,ND.0002,1883-1889,1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0


In [38]:
Shark_evol_anual.dropna(subset='Year', inplace=True) # subset somente vai dropar as linhas que tenham valores nulos na coluna Year
Shark_evol_anual['Year'] = Shark_evol_anual['Year'].astype(int)


In [39]:
Shark_evol_anual['Year'].sort_values(ascending=True)     # 725 336 214 493

6176       5
6175      77
6177     214
6178     336
6179     493
        ... 
30      2018
29      2018
28      2018
39      2018
0       2018
Name: Year, Length: 6293, dtype: int32

In [40]:
# Tentativa de trocar o sinal dos anos antes de Cristo
#dataframe.loc[dataframe['coluna'] == 'valor', 'coluna preencher'] = 'valor preencher'
#Shark_evol_anual.loc[Shark_evol_anual['Year'] == '214'] = '-214'

In [41]:
Shark_evol_anual

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0
6300,ND.0002,1883-1889,1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0


In [42]:
Shark_evol_anual['Country'].describe()

count     6243
unique     212
top        USA
freq      2228
Name: Country, dtype: object

In [43]:
# Verifica as linhas nulas na coluna Country,   50 rows × 22 columns
# Substituir por Alto Mar, considerando que nestes casos estão em alto mar entre um lugar e outro...
# Solucao Mayumi - sharkattack_main["Update Countries"] = sharkattack_main['Country'].map(lambda country: "IN BETWEEN COUNTRIES OR HIGH SEA" if country != country else country)

#Shark_evol_anual['Country'] = Shark_evol_anual['Country'].map(lambda  )      incompleto

Shark_evol_anual['Country'] = Shark_evol_anual['Country'].fillna('high seas')

In [44]:
Shark_evol_anual.count()       # coluna Country sem campos nulos  

Case Number               6292
Date                      6293
Year                      6293
Type                      6289
Country                   6293
Area                      5839
Location                  5754
Activity                  5749
Name                      6084
Sex                       5728
Age                       3471
Injury                    6265
Fatal (Y/N)               5754
Time                      2948
Species                   3464
Investigator or Source    6276
pdf                       6293
href formula              6292
href                      6293
Case Number.1             6293
Case Number.2             6293
original order            6293
dtype: int64

In [45]:
Shark_evol_anual['Country'] = Shark_evol_anual['Country'].str.upper().str.strip()

In [46]:
Shark_evol_anual

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0
6300,ND.0002,1883-1889,1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0


In [47]:
Shark_evol_anual['Country'] = Shark_evol_anual['Country'].str.replace(r'.*(SEA|OCEAN|\(|\)|RIVER|GULF|BETWEEN|COAST|/|CHANNEL|PACIFIC|"|\.|\?|\d|BOAT|SHARK).*', 'HIGH SEAS').fillna('HIGH SEAS')

C:\Users\carlo\AppData\Local\Temp\ipykernel_23944\4136702440.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Shark_evol_anual['Country'] = Shark_evol_anual['Country'].str.replace(r'.*(SEA|OCEAN|\(|\)|RIVER|GULF|BETWEEN|COAST|/|CHANNEL|PACIFIC|"|\.|\?|\d|BOAT|SHARK).*', 'HIGH SEAS').fillna('HIGH SEAS')


In [48]:
# Coluna Country limpa, coluna Year limpa
Shark_evol_anual['Country'].unique()        # Consulta valores unicos na coluna Country
#[print(s) for s in Shark_evol_anual['Country'].unique()]              # Consulta valores unicos na coluna Country ordenada por linha

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', 'HIGH SEAS', 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'COMOROS', 'REUNION', 'FRENCH POLYNESIA',
       'UNITED KINGDOM', 'UNITED ARAB EMIRATES', 'PHILIPPINES',
       'INDONESIA', 'CHINA', 'COLUMBIA', 'CAPE VERDE', 'FIJI',
       'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE',
       'PUERTO RICO', 'ITALY', 'GREECE', 'FRANCE', 'PAPUA NEW GUINEA',
       'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL', 'DIEGO GARCIA',
       'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES', 'GUAM',
       'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND', 'CANADA',
       'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA', 'RUSSIA',
       'TURKS & CAICOS', 'AZORES', 'SOUTH KOREA', 'MALTA', 'VIETNAM',
       'MADAGASCAR', 'PANAMA', 'SOMALIA',

In [49]:
Shark_evol_anual.columns = Shark_evol_anual.columns.str.strip()        # Remove espaços vazios antes e depois dos nomes das colunas

In [50]:
Shark_evol_anual['Sex'] = Shark_evol_anual['Sex'].str.strip()

In [51]:
Shark_evol_anual['Sex'].unique()

array(['F', 'M', nan, 'lli', 'N', '.'], dtype=object)

In [52]:
Shark_evol_anual['Sex'] = Shark_evol_anual['Sex'].str.replace(r'.*(lli|N|\.).*', 'NO INF').fillna('NO INF')

C:\Users\carlo\AppData\Local\Temp\ipykernel_23944\1607232364.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Shark_evol_anual['Sex'] = Shark_evol_anual['Sex'].str.replace(r'.*(lli|N|\.).*', 'NO INF').fillna('NO INF')


In [53]:
Shark_evol_anual['Sex'].info()   # Finalizado limpeza da coluna Sex

<class 'pandas.core.series.Series'>
Int64Index: 6293 entries, 0 to 6301
Series name: Sex
Non-Null Count  Dtype 
--------------  ----- 
6293 non-null   object
dtypes: object(1)
memory usage: 98.3+ KB


Shark_evol_anual['Sex'].to_csv('Versao5.csv', sep=',', index=False)

In [54]:
Shark_evol_anual.count()

Case Number               6292
Date                      6293
Year                      6293
Type                      6289
Country                   6293
Area                      5839
Location                  5754
Activity                  5749
Name                      6084
Sex                       6293
Age                       3471
Injury                    6265
Fatal (Y/N)               5754
Time                      2948
Species                   3464
Investigator or Source    6276
pdf                       6293
href formula              6292
href                      6293
Case Number.1             6293
Case Number.2             6293
original order            6293
dtype: int64

In [55]:
# Iniciar tratamento da coluna Fatal (Y/N) 
Shark_evol_anual.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [56]:
Shark_evol_anual['Fatal (Y/N)'] = Shark_evol_anual['Fatal (Y/N)'].str.strip().str.upper()

In [57]:
Shark_evol_anual['Fatal (Y/N)'].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017'], dtype=object)

In [58]:
Shark_evol_anual['Fatal (Y/N)'] = Shark_evol_anual['Fatal (Y/N)'].str.replace(r'.*(M|UNKNOWN|2017).*', 'NO INF').fillna('NO INF')

C:\Users\carlo\AppData\Local\Temp\ipykernel_23944\1698444571.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Shark_evol_anual['Fatal (Y/N)'] = Shark_evol_anual['Fatal (Y/N)'].str.replace(r'.*(M|UNKNOWN|2017).*', 'NO INF').fillna('NO INF')


In [59]:
Shark_evol_anual['Fatal (Y/N)'].unique()       # Coluna Fatal (Y/N) finalizado limipeza

array(['N', 'Y', 'NO INF'], dtype=object)

In [60]:
Shark_evol_anual['Fatal (Y/N)'].describe()

count     6293
unique       3
top          N
freq      4293
Name: Fatal (Y/N), dtype: object

In [61]:
# Iniciar tratamento coluna href formula
Shark_evol_anual['href formula'].str.startswith('http').value_counts()

True     6289
False       3
Name: href formula, dtype: int64

In [62]:
            #shark_no_empty.loc[(shark_no_empty['Year'] == 0)]['Date'].str.replace('.*?(\d+).*?',r'\1',regex=True)

            #Shark_evol_anual.loc[(Shark_evol_anual['href formula'] == Nan,'href formula'] = Shark_evol_anual.loc[(Shark_evol_anual['href formula'] == 0)]['href']
            #Shark_evol_anual.loc[(Shark_evol_anual['href formula'].isna()), 'href formula'] = Shark_evol_anual['href']


#Ver este código com a Raiana
#Shark_evol_anual.loc[(Shark_evol_anual['href formula'].isna()),'href formula'] = Shark_evol_anual.loc[(Shark_evol_anual['href formula'].isna())]['href'].str.extract()[0]


            #Shark_evol_anual.loc[Shark_evol_anual['href formula'].isnull()]

Shark_evol_anual['href formula'].to_csv('Versao6 href formula.csv', sep=',', index=False)

In [63]:
Shark_evol_anual['Country'].unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', 'HIGH SEAS', 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'COMOROS', 'REUNION', 'FRENCH POLYNESIA',
       'UNITED KINGDOM', 'UNITED ARAB EMIRATES', 'PHILIPPINES',
       'INDONESIA', 'CHINA', 'COLUMBIA', 'CAPE VERDE', 'FIJI',
       'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE',
       'PUERTO RICO', 'ITALY', 'GREECE', 'FRANCE', 'PAPUA NEW GUINEA',
       'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL', 'DIEGO GARCIA',
       'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES', 'GUAM',
       'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND', 'CANADA',
       'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA', 'RUSSIA',
       'TURKS & CAICOS', 'AZORES', 'SOUTH KOREA', 'MALTA', 'VIETNAM',
       'MADAGASCAR', 'PANAMA', 'SOMALIA',

Chat GPT

from geopy.geocoders import Nominatim
import pycountry

geolocator = Nominatim(user_agent="my-application")

# Criar um dicionário para armazenar as coordenadas de todos os países
coordenadas_paises = {}

# Loop através de todos os países do mundo
for pais in pycountry.countries:
    nome_pais = pais.name
    codigo_iso = pais.alpha_2
    
    # Obter as coordenadas de latitude e longitude do país usando o nome ou o código ISO
    try:
        location = geolocator.geocode(nome_pais)
    except:
        location = geolocator.geocode(codigo_iso)
        
    if location is not None:
        coordenadas_paises[nome_pais] = (location.latitude, location.longitude)
    else:
        coordenadas_paises[nome_pais] = (None, None)

# Imprimir as coordenadas de todos os países
for pais, coordenadas in coordenadas_paises.items():
    print(pais, coordenadas)


In [64]:
# Criar nova coluna vazia
Shark_evol_anual['Country_Localiz'] = ""

# Reorganizar as colunas
Shark_evol_anual = Shark_evol_anual[['Year', 'Country', 'Country_Localiz', 'Sex', 'Fatal (Y/N)', 'href formula',
                                     'Case Number', 'Date', 'Type', 'Area', 'Location', 'Activity',
                                     'Name', 'Age', 'Injury', 'Time', 'Species', 'Investigator or Source',
                                     'pdf', 'href', 'Case Number.1', 'Case Number.2', 'original order']]

In [65]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [66]:
paises_unicos = Shark_evol_anual['Country'].unique()
paises_unicos = pd.DataFrame(paises_unicos)

In [67]:
paises_unicos = paises_unicos.rename(columns={0:'Pais_Unico'})

In [68]:
paises_unicos

,Pais_Unico
0,USA
1,AUSTRALIA
2,MEXICO
3,BRAZIL
4,ENGLAND
...,...
158,SWEDEN
159,ROATAN
160,DJIBOUTI
161,BAHREIN


In [69]:
paises_unicos['Geolocaliza'] = paises_unicos['Pais_Unico'].apply(geocode)

In [70]:
paises_unicos['Pais_Lat'] = paises_unicos['Geolocaliza'].apply(lambda loc: loc.latitude if loc else None)
paises_unicos['Pais_Long'] = paises_unicos['Geolocaliza'].apply(lambda loc: loc.longitude if loc else None)


In [71]:
paises_unicos['Pais_Lat_Long'] = paises_unicos.apply(lambda x: str(x['Pais_Lat']) + ',' +str(x['Pais_Long']), axis=1)


In [72]:
paises_unicos = paises_unicos.rename(columns={'Pais_Unico':'Country', 'Pais_Lat_Long':'Country_Localiz'})

In [73]:
paises_unicos

,Country,Geolocaliza,Pais_Lat,Pais_Long,Country_Localiz
0,USA,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882,"39.7837304,-100.445882"
1,AUSTRALIA,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000,"-24.7761086,134.755"
2,MEXICO,"(México, (23.6585116, -102.0077097))",23.658512,-102.007710,"23.6585116,-102.0077097"
3,BRAZIL,"(Brasil, (-10.3333333, -53.2))",-10.333333,-53.200000,"-10.3333333,-53.2"
4,ENGLAND,"(England, United Kingdom, (52.5310214, -1.2649...",52.531021,-1.264906,"52.5310214,-1.2649062"
...,...,...,...,...,...
158,SWEDEN,"(Sverige, (59.6749712, 14.5208584))",59.674971,14.520858,"59.6749712,14.5208584"
159,ROATAN,"(Roatán, Islas de la Bahía, 33166, Honduras, (...",16.327423,-86.536481,"16.3274234,-86.53648145696454"
160,DJIBOUTI,"(Djibouti جيبوتي, (11.8145966, 42.8453061))",11.814597,42.845306,"11.8145966,42.8453061"
161,BAHREIN,"(البحرين, (26.1551249, 50.5344606))",26.155125,50.534461,"26.1551249,50.5344606"


In [74]:
teste_total = pd.merge(Shark_evol_anual, paises_unicos, on='Country', how='left')
pd.DataFrame(teste_total)

,Year,Country,Country_Localiz_x,Sex,Fatal (Y/N),href formula,Case Number,Date,Type,Area,...,Investigator or Source,pdf,href,Case Number.1,Case Number.2,original order,Geolocaliza,Pais_Lat,Pais_Long,Country_Localiz_y
0,2018,USA,,F,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,25-Jun-2018,Boating,California,...,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882,"39.7837304,-100.445882"
1,2018,USA,,F,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,18-Jun-2018,Unprovoked,Georgia,...,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882,"39.7837304,-100.445882"
2,2018,USA,,M,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,09-Jun-2018,Invalid,Hawaii,...,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882,"39.7837304,-100.445882"
3,2018,AUSTRALIA,,M,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,08-Jun-2018,Unprovoked,New South Wales,...,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000,"-24.7761086,134.755"
4,2018,MEXICO,,M,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,04-Jun-2018,Provoked,Colima,...,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,"(México, (23.6585116, -102.0077097))",23.658512,-102.007710,"23.6585116,-102.0077097"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6288,1903,AUSTRALIA,,M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,Before 1903,Unprovoked,Western Australia,...,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000,"-24.7761086,134.755"
6289,1903,AUSTRALIA,,M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,Before 1903,Unprovoked,Western Australia,...,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000,"-24.7761086,134.755"
6290,1905,USA,,M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,1900-1905,Unprovoked,North Carolina,...,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882,"39.7837304,-100.445882"
6291,1889,PANAMA,,M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,1883-1889,Unprovoked,NaN,...,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,"(Panamá, (8.559559, -81.1308434))",8.559559,-81.130843,"8.559559,-81.1308434"


In [75]:
teste_total = teste_total.drop('Country_Localiz_x', axis=1)

In [76]:
teste_total.columns

Index(['Year', 'Country', 'Sex', 'Fatal (Y/N)', 'href formula', 'Case Number',
       'Date', 'Type', 'Area', 'Location', 'Activity', 'Name', 'Age', 'Injury',
       'Time', 'Species', 'Investigator or Source', 'pdf', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Geolocaliza',
       'Pais_Lat', 'Pais_Long', 'Country_Localiz_y'],
      dtype='object')

In [77]:
teste_total = teste_total.rename(columns={'Country_Localiz_y':'Pais_Lat_Long'})
teste_total.columns

Index(['Year', 'Country', 'Sex', 'Fatal (Y/N)', 'href formula', 'Case Number',
       'Date', 'Type', 'Area', 'Location', 'Activity', 'Name', 'Age', 'Injury',
       'Time', 'Species', 'Investigator or Source', 'pdf', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Geolocaliza',
       'Pais_Lat', 'Pais_Long', 'Pais_Lat_Long'],
      dtype='object')

In [78]:
teste_total = teste_total[['Year', 'Country', 'Pais_Lat_Long', 'Sex', 'Fatal (Y/N)', 'href formula', 'Case Number',
       'Date', 'Type', 'Area', 'Location', 'Activity', 'Name', 'Age', 'Injury',
       'Time', 'Species', 'Investigator or Source', 'pdf', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Geolocaliza',
       'Pais_Lat', 'Pais_Long']]
teste_total.columns

Index(['Year', 'Country', 'Pais_Lat_Long', 'Sex', 'Fatal (Y/N)',
       'href formula', 'Case Number', 'Date', 'Type', 'Area', 'Location',
       'Activity', 'Name', 'Age', 'Injury', 'Time', 'Species',
       'Investigator or Source', 'pdf', 'href', 'Case Number.1',
       'Case Number.2', 'original order', 'Geolocaliza', 'Pais_Lat',
       'Pais_Long'],
      dtype='object')

In [79]:
teste_total

,Year,Country,Pais_Lat_Long,Sex,Fatal (Y/N),href formula,Case Number,Date,Type,Area,...,Species,Investigator or Source,pdf,href,Case Number.1,Case Number.2,original order,Geolocaliza,Pais_Lat,Pais_Long
0,2018,USA,"39.7837304,-100.445882",F,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,25-Jun-2018,Boating,California,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
1,2018,USA,"39.7837304,-100.445882",F,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,18-Jun-2018,Unprovoked,Georgia,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
2,2018,USA,"39.7837304,-100.445882",M,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,09-Jun-2018,Invalid,Hawaii,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
3,2018,AUSTRALIA,"-24.7761086,134.755",M,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,08-Jun-2018,Unprovoked,New South Wales,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000
4,2018,MEXICO,"23.6585116,-102.0077097",M,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,04-Jun-2018,Provoked,Colima,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,"(México, (23.6585116, -102.0077097))",23.658512,-102.007710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6288,1903,AUSTRALIA,"-24.7761086,134.755",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,Before 1903,Unprovoked,Western Australia,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000
6289,1903,AUSTRALIA,"-24.7761086,134.755",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,Before 1903,Unprovoked,Western Australia,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000
6290,1905,USA,"39.7837304,-100.445882",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,1900-1905,Unprovoked,North Carolina,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
6291,1889,PANAMA,"8.559559,-81.1308434",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,1883-1889,Unprovoked,NaN,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,"(Panamá, (8.559559, -81.1308434))",8.559559,-81.130843


#shark_no_empty.to_csv('Versao1.csv', sep=',', index=False)

In [80]:
shark = teste_total

In [81]:
shark.loc[6292, 'Pais_Lat_Long'] = '-20.140329, -144.886706'

In [82]:
shark

,Year,Country,Pais_Lat_Long,Sex,Fatal (Y/N),href formula,Case Number,Date,Type,Area,...,Species,Investigator or Source,pdf,href,Case Number.1,Case Number.2,original order,Geolocaliza,Pais_Lat,Pais_Long
0,2018,USA,"39.7837304,-100.445882",F,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,25-Jun-2018,Boating,California,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
1,2018,USA,"39.7837304,-100.445882",F,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,18-Jun-2018,Unprovoked,Georgia,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
2,2018,USA,"39.7837304,-100.445882",M,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,09-Jun-2018,Invalid,Hawaii,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
3,2018,AUSTRALIA,"-24.7761086,134.755",M,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,08-Jun-2018,Unprovoked,New South Wales,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000
4,2018,MEXICO,"23.6585116,-102.0077097",M,N,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,04-Jun-2018,Provoked,Colima,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,"(México, (23.6585116, -102.0077097))",23.658512,-102.007710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6288,1903,AUSTRALIA,"-24.7761086,134.755",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,Before 1903,Unprovoked,Western Australia,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000
6289,1903,AUSTRALIA,"-24.7761086,134.755",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,Before 1903,Unprovoked,Western Australia,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,"(Australia, (-24.7761086, 134.755))",-24.776109,134.755000
6290,1905,USA,"39.7837304,-100.445882",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,1900-1905,Unprovoked,North Carolina,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
6291,1889,PANAMA,"8.559559,-81.1308434",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,1883-1889,Unprovoked,NaN,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,"(Panamá, (8.559559, -81.1308434))",8.559559,-81.130843


shark.to_csv('ItsTheFinalCountDown.csv', sep=',', index=False)

In [85]:
plotar_df = shark[['Year','Country','Pais_Lat',
                 'Pais_Long','Pais_Lat_Long','Sex','Fatal (Y/N)','href formula']]
pd.DataFrame(plotar_df)

,Year,Country,Pais_Lat,Pais_Long,Pais_Lat_Long,Sex,Fatal (Y/N),href formula
0,2018,USA,39.783730,-100.445882,"39.7837304,-100.445882",F,N,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2018,USA,39.783730,-100.445882,"39.7837304,-100.445882",F,N,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2018,USA,39.783730,-100.445882,"39.7837304,-100.445882",M,N,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2018,AUSTRALIA,-24.776109,134.755000,"-24.7761086,134.755",M,N,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2018,MEXICO,23.658512,-102.007710,"23.6585116,-102.0077097",M,N,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...,...,...,...,...,...,...
6288,1903,AUSTRALIA,-24.776109,134.755000,"-24.7761086,134.755",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...
6289,1903,AUSTRALIA,-24.776109,134.755000,"-24.7761086,134.755",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...
6290,1905,USA,39.783730,-100.445882,"39.7837304,-100.445882",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...
6291,1889,PANAMA,8.559559,-81.130843,"8.559559,-81.1308434",M,Y,http://sharkattackfile.net/spreadsheets/pdf_di...


In [89]:
pip install xlwt

     -------------------------------------- 100.0/100.0 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [90]:
plotar_df.to_excel('PlotarTableau.xls', index=False)

C:\Users\carlo\AppData\Local\Temp\ipykernel_23944\894051926.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  plotar_df.to_excel('PlotarTableau.xls', index=False)


In [86]:
plotar_df['Country'].value_counts()    # Mas quantos foram em cada ano? para poder plotar o gráfico?

USA                         2228
AUSTRALIA                   1336
SOUTH AFRICA                 577
HIGH SEAS                    170
PAPUA NEW GUINEA             132
                            ... 
NORTHERN MARIANA ISLANDS       1
JAVA                           1
BAY OF BENGAL                  1
SLOVENIA                       1
KOREA                          1
Name: Country, Length: 163, dtype: int64

In [87]:
# agrupando por Year e Country e contando as ocorrências
plotar2 = plotar_df.groupby(['Year', 'Country']).size().reset_index(name='Count').transpose()

pd.DataFrame(plotar2)


,0,1,2,3,4,5,6,7,8,9,...,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897
Year,5,77,214,336,493,500,725,1543,1554,1555,...,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018
Country,AUSTRALIA,HIGH SEAS,HIGH SEAS,GREECE,GREECE,MEXICO,ITALY,VENEZUELA,FRANCE,HIGH SEAS,...,BRAZIL,COSTA RICA,ECUADOR,ENGLAND,MALDIVES,MEXICO,NEW CALEDONIA,SOUTH AFRICA,THAILAND,USA
Count,1,1,1,1,1,1,1,1,1,1,...,5,1,1,1,1,1,2,4,2,10


In [ ]:
import pandas as pd
import bar_chart_race as bcr

# Carregar os dados
plotar_df = pd.read_csv('data.csv')

# Contar o número de ocorrências de cada país em cada ano
df_counts = plotar_df.groupby(['Year', 'Country']).size().reset_index(name='Count')

# Gerar o gráfico animado
bcr.bar_chart_race(
    plotar_df=df_counts,
    title='Número de ocorrências de países por ano',
    orientation='h',
    sort='desc',
    n_bars=10,
    fixed_order=False,
    steps_per_period=10,
    period_length=500,
    interpolate_period=False
)


FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
import ffmpeg
bcr.bar_chart_race(plotar3)

TypeError: Cannot interpolate with all object-dtype columns in the DataFrame. Try setting at least one column to a numeric dtype.

import matplotlib; print(matplotlib.__version__, matplotlib.__file__)